In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
import sys

In [3]:
with open('../boston_housing0.pkl','rb') as handle:
    train_set, train_set_normalised, val_set_normalised, test_set, train_mean, train_sd = pickle.load(handle)

Xn = train_set_normalised[:,:-1]
Yn = np.expand_dims(train_set_normalised[:,-1],axis=1)

In [44]:
class variational_GP(nn.Module):
    
    def __init__(self, Xn, Yn):   # the GP takes the training data as arguments, in the form of numpy arrays, with the correct dimensions
        
        super().__init__()
        # initialise hyperparameters
        self.Xn = torch.tensor(Xn).type(torch.FloatTensor)
        self.Yn = torch.tensor(Yn).type(torch.FloatTensor)
        self.Xm = nn.Parameter(self.Xn[:20].type(torch.FloatTensor)) # still have to select randomly!!! Not yet because deterministic makes debugging easier
        self.no_inputs = Xn.shape[1]
        self.logsigmaf2 = nn.Parameter(torch.Tensor([0])) # function variance
        self.logl2 = nn.Parameter(torch.zeros(self.no_inputs)) # horizontal length scales
        self.logsigman2 = nn.Parameter(torch.Tensor([0])) # noise variance
        
    def get_K(self,inputs1,inputs2):
        
        inputs1_col = torch.unsqueeze(inputs1.transpose(0,1), 2)
        inputs2_row = torch.unsqueeze(inputs2.transpose(0,1), 1)
        squared_distances = (inputs1_col - inputs2_row)**2        
        length_factors = (1/(2*torch.exp(self.logl2))).reshape(self.no_inputs,1,1)
        K = torch.exp(self.logsigmaf2) * torch.exp(-torch.sum(length_factors * squared_distances, 0))
        return(K)
    
    def Fv_inv(self): # All the necessary arguments are instance variables, so no need to pass them

        Knn = self.get_K(self.Xn,self.Xn)
        Kmm = self.get_K(self.Xm,self.Xm)
        Kmn = self.get_K(self.Xm,self.Xn)
        Knm = Kmn.transpose(0,1)
        # Compute first term (log marginal likelihood) NEED TO CHECK WHETHER THIS DERIVATION IS CORRECT
        M = Kmm + 1/torch.exp(self.logsigman2) * torch.mm(Kmn,Knm)
        L_M = torch.cholesky(M)
        b_M, _ = torch.gesv(torch.mm(Kmn,self.Yn),L_M)
        log_marg = (-Xn.shape[0]/2*torch.log(2*np.pi*torch.exp(self.logsigman2)) 
                    - 1/torch.exp(self.logsigman2)*torch.mm(self.Yn.transpose(0,1),self.Yn) 
                    - 1/torch.exp(self.logsigman2)**2 * torch.mm(b_M.transpose(0,1),b_M))
        # Compute second term (trace, regularizer) USING INVERSE TEMPORARILY. NEED TO CHECK WHETHER WE CAN DO GRADIENTS THROUGH NON-SQUARE SYSTEM OF EQUATIONS
        reg = - 1/torch.exp(self.logsigman2) * torch.trace(Knn - torch.mm(torch.mm(Knm, torch.inverse(Kmm)), Kmn))

        return(log_marg + reg)
    
    def optimize_parameters(self,no_iters,method):
        
        # Set criterion and optimizer FOR NOW I'M GONNA USE ADAM ONLY
        '''if method == 'BFGS':
            optimizer = optim.LBFGS(self.parameters(), lr=1)  
        elif method == 'Adam':
            optimizer = optim.Adam(self.parameters(), lr=0.001)
        else: 
            sys.exit('method must be either \'BFGS\' or \'Adam\'') # An exception would be better
        
        for iteration in range(no_iters):
            optimizer.zero_grad()
            loss = self.Fv_inv() # Forward
            loss.backward() # Backward
            optimizer.step() # Optimize'''
            
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        for iteration in range(no_iters):
            optimizer.zero_grad()
            loss = -self.Fv_inv() # Forward. Negate because we want to maximize
            loss.backward() # Backward
            optimizer.step() # Optimize
            
            if iteration%50 == 0:
                print(iteration,-loss)


In [45]:
myGP = variational_GP(Xn, Yn)

After optimization, the variational lower bound has been maximized and the parameters have changed.

### Before:

In [46]:
myGP.logl2

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

In [47]:
myGP.logsigmaf2

Parameter containing:
tensor([0.], requires_grad=True)

In [48]:
myGP.logsigman2

Parameter containing:
tensor([0.], requires_grad=True)

In [49]:
myGP.Xm

Parameter containing:
tensor([[-4.1617e-01, -4.8207e-01, -1.1784e+00, -2.7964e-01, -8.3184e-01,
          5.6842e-02, -1.8585e+00,  6.8051e-01, -6.4458e-01,  1.2854e-01,
         -7.3106e-01,  2.1191e-01, -7.4717e-01],
        [ 6.4197e-01, -4.8207e-01,  1.0148e+00, -2.7964e-01,  6.4102e-01,
         -1.0660e-01,  1.1091e+00, -1.2244e+00,  1.6485e+00,  1.5426e+00,
          7.9477e-01,  1.1444e-01, -4.3881e-01],
        [-4.1075e-01, -4.8207e-01,  3.9342e-01,  3.5760e+00, -5.6652e-02,
          1.1272e-01,  8.3674e-01, -1.9121e-01, -5.2992e-01, -7.9420e-01,
         -9.6224e-01,  4.1126e-01, -3.0223e-01],
        [ 2.4395e+00, -4.8207e-01,  1.0148e+00, -2.7964e-01,  1.1750e+00,
         -1.3178e+00,  9.6577e-01, -9.7308e-01,  1.6485e+00,  1.5426e+00,
          7.9477e-01,  4.4526e-01,  1.0030e+00],
        [-4.1742e-01, -4.8207e-01,  2.3697e-01, -2.7964e-01, -1.0299e+00,
         -8.4250e-02, -5.5033e-01,  5.8610e-01, -5.2992e-01, -6.3198e-02,
          1.0121e-01,  3.3230e-01, -4.3145

In [50]:
myGP.Fv_inv()

tensor([[-1175.5045]], grad_fn=<AddBackward0>)

### During:

In [51]:
myGP.optimize_parameters(1000,'Adam')

0 tensor([[-1175.5045]], grad_fn=<NegBackward>)
50 tensor([[-1117.4366]], grad_fn=<NegBackward>)
100 tensor([[-1067.2606]], grad_fn=<NegBackward>)
150 tensor([[-1024.8361]], grad_fn=<NegBackward>)
200 tensor([[-989.4432]], grad_fn=<NegBackward>)
250 tensor([[-959.8486]], grad_fn=<NegBackward>)
300 tensor([[-934.8455]], grad_fn=<NegBackward>)
350 tensor([[-913.4968]], grad_fn=<NegBackward>)
400 tensor([[-895.0981]], grad_fn=<NegBackward>)
450 tensor([[-879.1000]], grad_fn=<NegBackward>)
500 tensor([[-865.0749]], grad_fn=<NegBackward>)
550 tensor([[-852.7040]], grad_fn=<NegBackward>)
600 tensor([[-841.7478]], grad_fn=<NegBackward>)
650 tensor([[-832.0123]], grad_fn=<NegBackward>)
700 tensor([[-823.3337]], grad_fn=<NegBackward>)
750 tensor([[-815.5717]], grad_fn=<NegBackward>)
800 tensor([[-808.6042]], grad_fn=<NegBackward>)
850 tensor([[-802.3282]], grad_fn=<NegBackward>)
900 tensor([[-796.6624]], grad_fn=<NegBackward>)
950 tensor([[-791.5453]], grad_fn=<NegBackward>)


### After:

In [52]:
myGP.Fv_inv()

tensor([[-786.9269]], grad_fn=<AddBackward0>)

In [53]:
myGP.logl2

Parameter containing:
tensor([-0.4345,  0.3736,  0.8932, -1.2515,  0.3020,  0.5562,  1.1750,  0.9067,
         0.0639,  0.9367,  0.9518,  0.1942,  0.7956], requires_grad=True)

In [54]:
myGP.logsigmaf2

Parameter containing:
tensor([-0.6762], requires_grad=True)

In [55]:
myGP.logsigman2

Parameter containing:
tensor([0.6132], requires_grad=True)

In [56]:
myGP.Xm

Parameter containing:
tensor([[-4.0936e-01,  3.8031e-01, -7.7444e-01, -2.7968e-01, -1.0194e+00,
         -3.1617e-01, -9.9130e-01,  1.4040e+00, -5.0853e-01, -5.5842e-01,
         -4.0630e-02,  3.6491e-01, -3.3373e-01],
        [ 4.6075e-01, -4.8217e-01,  1.0173e+00, -2.7962e-01,  1.0092e+00,
         -9.2791e-01,  8.4620e-01, -9.1865e-01,  1.6901e+00,  1.5693e+00,
          7.9101e-01,  1.5224e-01, -2.3590e-01],
        [-3.9629e-01, -4.2300e-01, -1.7654e-01,  3.5760e+00, -4.5328e-01,
         -3.1476e-01,  2.1852e-01, -7.8552e-02, -5.1014e-01, -7.6552e-01,
         -3.0363e-01,  3.7151e-01,  3.4788e-01],
        [ 2.7636e+00, -1.1234e+00,  4.5490e-01,  6.4649e-01,  1.6336e+00,
         -1.6187e+00,  4.5393e-01, -4.2708e-01,  2.4173e+00,  2.1012e+00,
          2.4808e-01,  9.1641e-01,  6.4435e-01],
        [-4.0686e-01, -4.3723e-01,  5.5187e-02, -2.7966e-01, -8.2693e-01,
         -3.1143e-01, -1.0846e+00,  3.5556e-01, -6.2159e-01, -4.5610e-01,
         -2.2684e-01,  3.7449e-01, -3.2612

Next step: get posterior, predictive distribution, and plot how it changes during training, to see that the algorithm is working.